# Model serialization

## Import model

In [ ]:
from __future__ import annotations

import json

from IPython.display import JSON, Markdown, Math

from ampform_dpd.io import as_markdown_table, aslatex
from ampform_dpd.io.serialization.decay import to_decay

In [ ]:
with open("Lc2ppiK.json") as stream:
    MODEL_DEFINITION = json.load(stream)

In [ ]:
JSON(MODEL_DEFINITION)

## Construct `ThreeBodyDecay`

In [ ]:
def name_to_latex(name: str) -> str:
    latex = {
        "Lc": R"\Lambda_c^+",
        "pi": R"\pi^+",
        "K": "K^-",
        "p": "p",
    }.get(name)
    if latex is not None:
        return latex
    mass_str = name[1:].strip("(").strip(")")
    subsystem_letter = name[0]
    subsystem = {"D": "D", "K": "K", "L": R"\Lambda"}.get(subsystem_letter)
    if subsystem is None:
        return name
    return f"{subsystem}({mass_str})"

In [ ]:
DECAY = to_decay(MODEL_DEFINITION, to_latex=name_to_latex)
Math(aslatex(DECAY))

In [ ]:
Markdown(as_markdown_table(DECAY))